<a href="https://colab.research.google.com/github/Jang-jw/DeepLearning/blob/main/ex07_%EA%B0%9C%2C%EA%B3%A0%EC%96%91%EC%9D%B4_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
# 드라이브 연동
# 경로 이동 -> 딥러닝 폴더까지
%cd /content/drive/MyDrive/Colab Notebooks/DeepLearning
# 상대 경로로 데이터에 접근하기 위해 디렉토리 변경

/content/drive/MyDrive/Colab Notebooks/DeepLearning


In [28]:
# 추가) 현재 디렉토리(폴더 경로) 확인
%pwd

'/content/drive/MyDrive/Colab Notebooks/DeepLearning'

### 목표
 - 고양이, 개 이미지(사진)를 배열로 변환하는 작업을 진행해보자(이미지 데이터 전처리)
   - 압축된 형식의 배열로 저장: npz
   - 대용량의 데이터를 압축형태로 정리해두면 클라우드 환경에 쉽게 업로드, 다운로드 할 수 있게 됨(관리가 용이)
   - 저장소의 공간 적게 차지함

In [29]:
# 1. 압축 파일 불러와서 해제
# 2. 압축이 풀린 폴더들 안에 있는 이미지 접근
# 3. 이미지 한장씩 모두를 그림-> nparray
# 4. nparray 하나의 리스트에 묶어서 변수에 저장
# 5. 묶어져 있는 변수를 npz라고 하는 확장자로 연결하여 파일 형식으로 저장

In [30]:
# 환경 셋팅
import numpy as np
from zipfile import ZipFile  # 압축 해제시 사용
import os # 파일을 다룰때 사용
from PIL import Image # 이미지 다룰때 사용

In [31]:
# 1. 압축 폴더 압축 해제
# 해제할 압축파일 경로 설정
zip_file_path = '/content/drive/MyDrive/Colab Notebooks/DeepLearning/data/cats_and_dogs_filtered.zip'
# ZipFile() 파일 열기
with ZipFile(zip_file_path, 'r') as f:
    # './data/'에 해제
    f.extractall(path = './data/')

In [32]:
# data폴더에 압축 해제된 폴더 생성됐는지 확인 !
%ls ./data/cats_and_dogs_filtered/

test/  train/


In [33]:
# filtered 폴더명
# train 폴더 - cats, dogs
# test 폴더 - cats, dogs
# 각 4개의 폴더 경로 변수 저장
train_cats_dir = './data/cats_and_dogs_filtered/train/cats'
train_dogs_dir = './data/cats_and_dogs_filtered/train/dogs'
test_cats_dir = './data/cats_and_dogs_filtered/test/cats'
test_dogs_dir = './data/cats_and_dogs_filtered/test/dogs'

# 각 폴더에 있는 이미지 파일명 확인
# os.listdir(폴더경로) : 해당 폴더 경로에 있는 모든 파일명을 리스트 출력
train_cats_fn = os.listdir(train_cats_dir)
train_dogs_fn = os.listdir(train_dogs_dir)
test_cats_fn = os.listdir(test_cats_dir)
test_dogs_fn = os.listdir(test_dogs_dir)

# 각 변수의 이미지 개수 확인
print('훈련 고양이 이미지:', len(train_cats_fn))
print('훈련 개 이미지:', len(train_dogs_fn))
print('테스트 고양이 이미지:', len(test_cats_fn))
print('테스트 개 이미지:', len(test_dogs_fn))
# 훈련 이미지 총 2000장, 테스트 이미지 총 1000장

훈련 고양이 이미지: 1000
훈련 개 이미지: 1000
테스트 고양이 이미지: 500
테스트 개 이미지: 500


In [34]:
train_cats_dir

'./data/cats_and_dogs_filtered/train/cats'

In [35]:
train_cats_fn[0]

'cat.103.jpg'

In [36]:
# 이미지 1장씩에 접근해서 그림 => 크기조정 => nparray
# 예시 1장만 배열로 변환
# train cats 이미지 1장 접근

tmp_path=os.path.join(train_cats_dir,train_cats_fn[0])
tmp_path

'./data/cats_and_dogs_filtered/train/cats/cat.103.jpg'

In [37]:
img=Image.open(tmp_path)
np.array(img).shape

(386, 500, 3)

In [38]:
# image 클래스
# 추가) 이미지를 다루는 lib - 1. opencv -2 PIL
img=Image.open(tmp_path).resize((224,224)) # 정사각형, (320,320), (640,640)
np.array(img).shape

(224, 224, 3)

In [39]:
# 4 개 폴더명, 각 폴더 이미지 파일명 연결해서 전처리하는 함수를 생성
def prepro_imgs(folder_path,file_name,img_size=(224,224)):
    # 변환된 이미지 배열을 담는 빈 리스트
    img_list=[] # 1 장 변환시 append 를 통해 이미지 누적 저장

    # 각 파일 접근하면서 불러오기 => 크기조정 => nparray
    for i in file_name:
        # 폴더 경로 + 파일 경로 합치기
        path=os.path.join(folder_path,i)

        # 파일 오픈 + 리사이즈 (크기조정 => img_size=224,224)
        img=Image.open(path).resize(img_size)

        # array 변환 => img_list 누적
        img_list.append(np.array(img))

    return np.array(img_list) # 모델 연결할때 list 보다는 array 연산하는게 용이

In [40]:
# 함수 호출 실행 총 4 번
X_train_cats=prepro_imgs(train_cats_dir,train_cats_fn)
X_train_dogs=prepro_imgs(train_dogs_dir,train_dogs_fn)
X_test_cats=prepro_imgs(train_cats_dir,train_cats_fn)
X_test_dogs=prepro_imgs(train_dogs_dir,train_dogs_fn)

# 크기확인
print('훈련데이터:',X_train_cats.shape,X_train_dogs.shape)
print('테스트데이터:',X_test_cats.shape,X_test_dogs.shape)
# (이미지샘플개수,행,열,채널수)
# 채널수 3 => 컬러이미지
# 채널수 1 => 흑백이미지

훈련데이터: (1000, 224, 224, 3) (1000, 224, 224, 3)
테스트데이터: (1000, 224, 224, 3) (1000, 224, 224, 3)


In [41]:
# 이미지 배열 1 장 접근해보기
X_train_dogs[0] # 이미지 1 장은 3 차원으로 표현 (컬러이미지)

array([[[117, 115, 127],
        [115, 113, 126],
        [106, 103, 119],
        ...,
        [133, 135, 147],
        [132, 134, 144],
        [132, 132, 140]],

       [[119, 117, 128],
        [117, 115, 128],
        [108, 105, 120],
        ...,
        [134, 136, 148],
        [134, 136, 146],
        [135, 135, 143]],

       [[121, 119, 131],
        [119, 117, 130],
        [110, 108, 123],
        ...,
        [136, 138, 150],
        [135, 137, 146],
        [136, 136, 144]],

       ...,

       [[ 70,  75,  79],
        [ 73,  79,  82],
        [ 70,  78,  81],
        ...,
        [ 62,  71,  76],
        [ 71,  81,  85],
        [ 77,  87,  89]],

       [[ 70,  75,  79],
        [ 72,  77,  81],
        [ 73,  81,  84],
        ...,
        [ 76,  85,  90],
        [ 76,  85,  89],
        [ 71,  82,  83]],

       [[ 66,  71,  75],
        [ 65,  71,  74],
        [ 67,  76,  79],
        ...,
        [ 81,  90,  95],
        [ 64,  73,  77],
        [ 64,  74,  76]]

In [42]:
# train 끼리 병합
# 배열 병합하는 함수 : np.concatenate((대상,대상))
# 고양이 + 개
X_train=np.concatenate((X_train_cats,X_train_dogs))
X_test=np.concatenate((X_test_cats,X_test_dogs))
# 크기확인
print('훈련이미지 크기:',X_train.shape) # 고양이 + 개 2000장
print('테스트이미지 크기:',X_test.shape) # 고양이 + 개 총 1000장

훈련이미지 크기: (2000, 224, 224, 3)
테스트이미지 크기: (2000, 224, 224, 3)


In [43]:
# 답 데이터 생성
# 고양이 : 0 , 개 : 1
# train 0-1000 개 , 1-1000 개 => 2000 개
# test 0-500 개 , 1-500 개 => 1000 개
# [0 천개, 1 천개]
y_train=np.array([0]*1000 + [1]*1000) # 훈련 답
# 테스트 답 생성
y_test=np.array([0]*500 + [1]*500)
# 크기확인
print('훈련답:',y_train.shape)
print('테스트:',y_test.shape)

훈련답: (2000,)
테스트: (1000,)


In [44]:
# 이미지들 => 배열
# 배열 압축해서 저장
# npz 확장자 파일 생성됐는지 확인
np.savez_compressed('./data/npz_cats_vs_dogs.npz',
                    X_train=X_train,
                    X_test=X_test,
                    y_train=y_train,
                    y_test=y_test)